In [1]:
#     def get_LL(self,rate,Y):
#         '''log-likelihood function'''
#         loglik = np.sum(Y*np.log(rate)-rate)
#         return loglik
    
    ##################################################################################################################
    
#     # functions from the tutorial; will probably change these to simplify (?)
    
#     def qu(self,z):
#         '''the nonlinearity (w/ softplus)'''
#         qu = np.log1p(np.exp(z))
        
#     def lmb(self,beta0,beta,X):
#         '''conditional intensity function'''
#         z = beta0 + np.dot(X,beta)
#         l = self.qu(z)
#         return l
    
#     def penalty(self,alpha,beta):
#         '''penalty term'''
#         P = 0.5 * (1 - alpha) * np.linalg.norm(beta, 2) ** 2 + \
#         alpha * np.linalg.norm(beta, 1)
#         return P
    
#     def loss(self,beta0, beta, reg_lambda, X, y):
#         '''define objective function for elastic net'''
#         L = logL(beta0, beta, X, y)
#         P = self.penalty(beta)
#         J = -L + reg_lambda * P
#         return J
    
#     def grad_L2loss(self,beta0, beta, reg_lambda, X, y):
#         z = beta0 + np.dot(X, beta)
#         s = expit(z)
#         q = self.qu(z)
#         grad_beta0 = np.sum(s) - np.sum(y * s / q)
#         grad_beta = np.transpose(np.dot(np.transpose(s), X) -
#                                  np.dot(np.transpose(y * s / q), X)) + \
#         reg_lambda * (1 - alpha) * beta
#         return grad_beta0, grad_beta
    
#     def hessian_loss(beta0, beta, alpha, reg_lambda, X, y):
#         z = beta0 + np.dot(X, beta)
#         q = qu(z)
#         s = expit(z)
#         grad_s = s * (1-s)
#         grad_s_by_q = grad_s/q - s/(q * q)
#         hess_beta0 = np.sum(grad_s) - np.sum(y * grad_s_by_q)
#         hess_beta = np.transpose(np.dot(np.transpose(grad_s), X * X)
#                                 - np.dot(np.transpose(y * grad_s_by_q), X * X))\
#                                 + reg_lambda * (1-alpha)
#         return hess_beta0, hess_beta

In [ ]:
glm = GLM(distr='poisson', score_metric='pseudo_R2', reg_lambda=0.01)
XX = X_train.to_numpy();
yy = y_train.to_numpy(); yy=yy[:,0]
glm.fit(XX,yy)

yhat = glm.predict(X_test.to_numpy())
pseudo_R2 = glm.score(X_test.to_numpy(), y_test.to_numpy())
print('Pseudo R^2 is %.3f' % pseudo_R2)

In [ ]:
    ####################### DEPRECIATE #######################
#     def test_train(self,df,expr,spiketrain):
#         df.insert(loc=0, column='y', value=spiketrain, allow_duplicates=False)
#         mask = np.random.rand(len(df)) < 0.8
#         df_train = df[mask]
#         df_test = df[~mask]
#         # split into test and train, and booty
#         y_train, X_train = dmatrices(expr, df_train, return_type='dataframe')
#         y_test, X_test = dmatrices(expr, df_test, return_type='dataframe')
# #         print('Training data set length='+str(len(df_train)))
# #         print('Testing data set length='+str(len(df_test)))
#         return y_train, X_train, y_test, X_test
    
#     def test_train_arr(self,df,expr,spiketrain):
#         '''return training set as array (not df)'''
#         y_train, X_train, y_test, X_test = self.test_train(df,expr,spiketrain)
#         y_train_arr = y_train.to_numpy(); y_test_arr = y_test.to_numpy()
#         X_test_arr = X_test.to_numpy(); X_train_arr = X_train.to_numpy()
#         X_train_arr = X_train_arr[:,1:];  X_test_arr = X_test_arr[:,1:]
#         return y_train_arr, X_train_arr, y_test_arr, X_test_arr
    
    ####################### DEPRECIATE #######################

#### visualize data 

In [ ]:
fig, ax = plt.subplots(1, 1);
ax.plot(test_y[fold]); ax.plot(yhat);
ax.set_xlabel('time step'); ax.set_ylabel('y (smoothed rate)');

fig, ax = plt.subplots(1, 2, figsize=(10, 2))
_, bin_edges = np.histogram(y,120)
ax[0].plot(bin_edges, scipy.stats.norm.pdf(bin_edges, loc=y.mean(), scale=y.std()))
ax[0].set_title(r'Distribution of Rates')
ax[0].set_xlabel('rate (hz)')
ax[0].set_ylabel('hist')

sns.kdeplot(y, color='#fcb103', bw=.017,shade=True)
ax[1].set_title(r'Distribution of Rates')
ax[1].set_xlabel('rate (hz)')
ax[1].set_ylabel('probability');

#### visualize resutls

In [ ]:
plt.bar(['PE', 'P', 'E'], sse);

w_fit = res.x[1:]
b_fit = res.x[0]
y_hat = g.get_rate(X,w_fit,b_fit)

fig, ax = plt.subplots(1, 1, figsize=(20, 8))
ax.plot(y[0:10000], label='data');
ax.plot(smooth_fr_hat_test[0:10000],label='model');
ax.set_title(r'model vs. data')
ax.set_xlabel('time (s)')
ax.set_ylabel('rate (hz)');
ax.legend(loc="upper right");

In [ ]:
# preamble for the original code
# preamble
import scipy.io
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import pandas as pd
import statsmodels.api as sm
from patsy import dmatrices
import scipy.sparse as sps
import scipy.stats as stats
from pyglmnet import GLM, simulate_glm
import scipy as sp
import statistics

In [ ]:
   ## WANT TO COMBINE OBJECTIVE & GRADIENT FUNCTION
#     def grad(self,param,x,y):
#         '''compute the gradient of the loss fn'''
#         # these are the first order derivatives of the cost
#         # function with respect to [each of] the weights
#         M, n = x.shape
#         y_hat = np.exp(x @ param[1:] + param[0])
#         dw = (x.T @ (y_hat - y)) / M
#         db = (y_hat - y).mean() 
#         # we dont really need [db] if the bias term is included as a column of 1's
#         # which right now it is currently NOT
#         jac = dw; jac=np.append(jac,db);
#         return jac

    #### UPDATE THIS FUNCTION! ######
#     def gradient_descent(self,x, y, w_0, b_0, alpha, num_iter):
#         '''minimize loss function w/ gradient descent'''
#         w, b = w_0.copy(), b_0
#         hist = np.zeros(num_iter)
#         M, n = x.shape
#         for iter in range(num_iter):
#             dw, db = self.grad(x, y, w, b)
#             w -= alpha * dw
#             b -= alpha * db
#             hist[iter] = self.loss(x, y, w, b)
#         return w, b, hist